<a href="https://colab.research.google.com/github/iannjari/scrapbook/blob/main/Clean_notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Import dependencies for data import

Un-comment the code cell below to obtain the data. Blocks should be seperated to test whether they're individually working.

In [1]:
'''
import requests
import tarfile
from os import path

# Downloading the dataset

fname = 'airline_2m.tar.gz'
url = 'https://dax-cdn.cdn.appdomain.cloud/dax-airline/1.0.1/' + fname
r = requests.get(url)
open(fname , 'wb').write(r.content)

# Extracting the dataset
tar = tarfile.open(fname)
tar.extractall()
tar.close()

# Verifying the file was extracted properly
data_path = "airline_2m.csv"
path.exists(data_path)

import pandas as pd 
df = pd.read_csv(data_path, encoding = "ISO-8859-1",
                 dtype={'Div1Airport': str, 'Div1TailNum': str, 'Div2Airport': str, 'Div2TailNum': str})
df.shape

'''

'\nimport requests\nimport tarfile\nfrom os import path\n\n# Downloading the dataset\n\nfname = \'airline_2m.tar.gz\'\nurl = \'https://dax-cdn.cdn.appdomain.cloud/dax-airline/1.0.1/\' + fname\nr = requests.get(url)\nopen(fname , \'wb\').write(r.content)\n\n# Extracting the dataset\ntar = tarfile.open(fname)\ntar.extractall()\ntar.close()\n\n# Verifying the file was extracted properly\ndata_path = "airline_2m.csv"\npath.exists(data_path)\n\nimport pandas as pd \ndf = pd.read_csv(data_path, encoding = "ISO-8859-1",\n                 dtype={\'Div1Airport\': str, \'Div1TailNum\': str, \'Div2Airport\': str, \'Div2TailNum\': str})\ndf.shape\n\n'

Getting the features (these were obtained by reviewing the data by datatype, description from the US DoT(helped remove redudant features) on excel, removing unwanted columns, re-reading the data and converting them to the dictionary below)

In [2]:
import pandas as pd

'''path = "airline_2m.csv"
df = pd.read_csv(path, encoding = "ISO-8859-1",
                 dtype={'Div1Airport': str, 'Div1TailNum': str, 'Div2Airport': str, 'Div2TailNum': str})
df.shape'''

'path = "airline_2m.csv"\ndf = pd.read_csv(path, encoding = "ISO-8859-1",\n                 dtype={\'Div1Airport\': str, \'Div1TailNum\': str, \'Div2Airport\': str, \'Div2TailNum\': str})\ndf.shape'

In [3]:
featurecols= ['Year','Quarter',
'Month','DayofMonth','DayOfWeek','DOT_ID_Reporting_Airline','Flight_Number_Reporting_Airline',
'OriginAirportID','OriginState','DestAirportID','DestState','DepTime','Cancelled']
featurecols

['Year',
 'Quarter',
 'Month',
 'DayofMonth',
 'DayOfWeek',
 'DOT_ID_Reporting_Airline',
 'Flight_Number_Reporting_Airline',
 'OriginAirportID',
 'OriginState',
 'DestAirportID',
 'DestState',
 'DepTime',
 'Cancelled']

In [4]:
#commmented since featuredata CSV is available featuredata=df[featurecols]
'''
featuredata=featuredata.dropna()
featuredata.to_csv('featuredata.csv',index=False)
'''

"\nfeaturedata=featuredata.dropna()\nfeaturedata.to_csv('featuredata.csv',index=False)\n"

In [5]:
featuredata=pd.read_csv('featuredata.csv')
featuredata=featuredata.dropna()
featuredata

,Year,Quarter,Month,DayofMonth,DayOfWeek,DOT_ID_Reporting_Airline,Flight_Number_Reporting_Airline,OriginAirportID,OriginState,DestAirportID,DestState,DepTime,Cancelled
0,1998,1,1,2,5,19386,675,13487,MN,14869.0,UT,1659.0,0.0
1,2009,2,5,28,4,20437,671,13342,WI,13204.0,FL,1202.0,0.0
2,2013,2,6,29,6,20398,3297,11921,CO,11298.0,TX,1644.0,0.0
3,2010,3,8,31,2,19790,1806,12892,CA,11433.0,MI,1305.0,0.0
4,2006,1,1,15,7,20355,465,11618,NJ,11057.0,NC,1911.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
492931,1994,4,10,8,6,19805,201,11298,TX,13891.0,CA,1244.0,0.0
492932,2008,1,1,16,3,20409,1075,12478,NY,14524.0,VA,1155.0,0.0
492933,2008,1,1,21,1,19805,2495,13930,IL,13198.0,MO,1112.0,0.0
492934,2003,3,8,1,5,19386,147,13487,MN,11637.0,ND,2218.0,0.0


In [6]:
target_name = "Cancelled"
target = featuredata[target_name]
# note that all features are categorical in nature
data = featuredata.drop(columns=[target_name])

In [7]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(
    data,target, test_size=0.33, random_state=42)


In [8]:
target.value_counts()

0.0    492820
1.0       116
Name: Cancelled, dtype: int64

Class imbalance might be an issue with models modelled on this data

In [9]:
data.isnull().values.any()

False

Use a dummy classifier that predicts the majority class always as the base model

In [10]:
from sklearn.dummy import DummyClassifier
dummy_clf = DummyClassifier(strategy="most_frequent")
dummy_clf.fit(x_train, y_train)


dummy_clf.predict(x_test)
dummy_clf.score(x_test, y_test)

0.999729512076671

High accuracy,arising from the imbalanced classes in the target.

What's the performance of a logistic regression?

In [11]:
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import LogisticRegression

logistic_regression = make_pipeline(
    OneHotEncoder(handle_unknown = 'ignore'), LogisticRegression()
)


from sklearn.model_selection import cross_validate
cv_results = cross_validate(
    logistic_regression, x_train, y_train, cv=10,return_estimator=True)

cv_results


{'estimator': (Pipeline(memory=None,
           steps=[('onehotencoder',
                   OneHotEncoder(categories='auto', drop=None,
                                 dtype=<class 'numpy.float64'>,
                                 handle_unknown='ignore', sparse=True)),
                  ('logisticregression',
                   LogisticRegression(C=1.0, class_weight=None, dual=False,
                                      fit_intercept=True, intercept_scaling=1,
                                      l1_ratio=None, max_iter=100,
                                      multi_class='auto', n_jobs=None,
                                      penalty='l2', random_state=None,
                                      solver='lbfgs', tol=0.0001, verbose=0,
                                      warm_start=False))],
           verbose=False), Pipeline(memory=None,
           steps=[('onehotencoder',
                   OneHotEncoder(categories='auto', drop=None,
                                 dtype

In [12]:
array=cv_results["test_score"]
array.mean()

0.9997819946229974

Cross-validation score of 99.97 percent

How will it perform on unseen data?

In [13]:
logistic_regression.fit(x_train, y_train)
predictions=logistic_regression.predict(x_test)
logistic_regression.score(x_test,y_test)

0.999729512076671

Look at other metrics

In [14]:
from sklearn.metrics import confusion_matrix
c_matrix=confusion_matrix(y_test, predictions)
tn, fp, fn, tp = c_matrix.ravel()

precision= tp/(tp+fp)
misclassification= (fp+fn)/(tn+fn+tp+fp)
f_one=tp/(tp+0.5*(fp+fn))

print('Precision=',precision)
print('Misclassification=',misclassification)
print('F1 score=',f_one)

Precision= nan
Misclassification= 0.00027048792332896865
F1 score= 0.0


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: RuntimeWarning: invalid value encountered in long_scalars
  """


In [15]:
tn,fp,fn,tp

(162625, 0, 44, 0)

In [16]:
y_test.value_counts()

0.0    162625
1.0        44
Name: Cancelled, dtype: int64

Model learnt to predict most common class due to class imbalance

In [17]:
from imblearn.over_sampling import RandomOverSampler

ros = RandomOverSampler(random_state=42)

# fit predictor and target 

x_ros, y_ros = ros.fit_resample(x_train, y_train)

y_ros=pd.DataFrame(y_ros)

print('Original dataset shape', y_train.value_counts())
print('Resample dataset shape', y_ros.value_counts())

/usr/local/lib/python3.7/dist-packages/sklearn/externals/six.py:31: FutureWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.neighbors.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.neighbors. Anything that cannot be imported from sklearn.neighbors is now part of the private API.
  warnings.warn(message, FutureWarning)


Original dataset shape 0.0    330195
1.0        72
Name: Cancelled, dtype: int64
Resample dataset shape 1.0    330195
0.0    330195
dtype: int64


/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)


In [18]:
logistic_regression2 = make_pipeline(
    OneHotEncoder(handle_unknown = 'ignore'), LogisticRegression(max_iter=1000)
)


cv_results = cross_validate(
    logistic_regression2, x_ros, y_ros[0], cv=10,return_estimator=True)

cv_results

{'estimator': (Pipeline(memory=None,
           steps=[('onehotencoder',
                   OneHotEncoder(categories='auto', drop=None,
                                 dtype=<class 'numpy.float64'>,
                                 handle_unknown='ignore', sparse=True)),
                  ('logisticregression',
                   LogisticRegression(C=1.0, class_weight=None, dual=False,
                                      fit_intercept=True, intercept_scaling=1,
                                      l1_ratio=None, max_iter=1000,
                                      multi_class='auto', n_jobs=None,
                                      penalty='l2', random_state=None,
                                      solver='lbfgs', tol=0.0001, verbose=0,
                                      warm_start=False))],
           verbose=False), Pipeline(memory=None,
           steps=[('onehotencoder',
                   OneHotEncoder(categories='auto', drop=None,
                                 dtyp

In [19]:
array=cv_results["test_score"]
array.mean()

0.9994760671724284

Cross-validation score of 99.79 percent

How will it perform on unseen data?

In [20]:
logistic_regression2.fit(x_ros,y_ros[0])
predictions=logistic_regression2.predict(x_test)
logistic_regression2.score(x_test,y_test)

0.9988811635898666

In [21]:
c_matrix=confusion_matrix(y_test, predictions)
tn, fp, fn, tp = c_matrix.ravel()
precision= tp/(tp+fp)
misclassification= (fp+fn)/(tn+fn+tp+fp)
f_one=tp/(tp+0.5*(fp+fn))

print('Precision=',precision)
print('Misclassification=',misclassification)
print('F1 score=',f_one)

Precision= 0.0
Misclassification= 0.0011188364101334613
F1 score= 0.0


In [22]:
tp,tn,fp,fn

(0, 162487, 138, 44)

Try undersampling

In [23]:
# Training will be done on smaller dataset

# Create dataframe from oversampled data
df=pd.DataFrame(x_ros)
df['target']=y_ros
df

,0,1,2,3,4,5,6,7,8,9,10,11,target
0,2011,4,10,24,1,19790,2015,10397,GA,12451,FL,1452,0.0
1,2011,3,7,31,7,20366,4909,12448,MS,10397,GA,1928,0.0
2,2010,4,11,19,5,19393,240,12191,TX,13851,OK,827,0.0
3,2014,3,9,9,2,20398,3668,12448,MS,11298,TX,827,0.0
4,2019,3,9,23,1,19805,1079,14747,WA,11298,TX,1206,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
660385,2010,3,7,30,5,20304,6275,14747,WA,11884,WA,1259,1.0
660386,2009,2,6,25,4,20363,2453,10577,NY,11433,MI,1711,1.0
660387,2014,1,2,21,5,20355,1910,11057,NC,12953,NY,1240,1.0
660388,2012,3,8,9,4,20398,3801,14100,PA,13930,IL,1744,1.0


In [24]:
# Sample data
df2=df.sample(n=300000)
df2['target'].value_counts()

1.0    150035
0.0    149965
Name: target, dtype: int64

In [25]:
x,y = df2.drop(columns=['target']),df2['target']
x_train, x_test, y_train, y_test = train_test_split(x,y,stratify=y,random_state=42)

Try stacked classifiers

In [26]:
from sklearn.ensemble import (RandomForestClassifier, StackingClassifier, GradientBoostingClassifier)
from sklearn.linear_model import LogisticRegression
from  sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier

In [29]:


clf=StackingClassifier(estimators=[
            ("rf",RandomForestClassifier(n_estimators=10,random_state=42)),
            ("gb",GradientBoostingClassifier(n_estimators=10,random_state=42)),
            ("knn",KNeighborsClassifier(n_neighbors=5))],final_estimator=LogisticRegression())

In [30]:
pipeline = make_pipeline(
    OneHotEncoder(handle_unknown = 'ignore'), StackingClassifier(estimators=[
            ("rf",RandomForestClassifier(n_estimators=10,random_state=42)),
            ("gb",GradientBoostingClassifier(n_estimators=10,random_state=42)),
            ("knn",KNeighborsClassifier(n_neighbors=5))],final_estimator=LogisticRegression()))

pipeline.fit(x_train,y_train)
pipeline.score(x_test,y_test)

0.99996

In [31]:
preds=pipeline.predict(x_test)
c_matrix=confusion_matrix(y_test, preds)
tn, fp, fn, tp = c_matrix.ravel()
(tn, fp, fn, tp)

(37488, 3, 0, 37509)

In [32]:
precision= tp/(tp+fp)
misclassification= (fp+fn)/(tn+fn+tp+fp)
f_one=tp/(tp+0.5*(fp+fn))

print('Precision=',precision)
print('Misclassification=',misclassification)
print('F1 score=',f_one)

Precision= 0.9999200255918106
Misclassification= 4e-05
F1 score= 0.9999600111968648


Save this model in ONNX format

In [33]:
pip install skl2onnx

     |████████████████████████████████| 265 kB 5.1 MB/s 
     |████████████████████████████████| 12.3 MB 40.1 MB/s 
     |████████████████████████████████| 77 kB 6.0 MB/s 


In [34]:
x_train.columns=featurecols[:-1]
x_train.to_numpy()

array([[1994, 2, 4, ..., 12197.0, 'NY', 1605.0],
       [2010, 1, 2, ..., 11503.0, 'CO', 921.0],
       [1993, 2, 4, ..., 12339.0, 'IN', 1932.0],
       ...,
       [1992, 1, 2, ..., 10397.0, 'GA', 1459.0],
       [2014, 1, 2, ..., 12953.0, 'NY', 1240.0],
       [2010, 3, 7, ..., 11884.0, 'WA', 1259.0]], dtype=object)

In [54]:
import skl2onnx 
from skl2onnx import convert_sklearn
from skl2onnx.common.data_types import FloatTensorType, StringTensorType



initial_types =  [('feature_input', FloatTensorType([None, 10])),
                  ('feature_input', StringTensorType([None,2]))]

onx = convert_sklearn(pipeline,
                      initial_types=
                      initial_types)

with open("stacked_clf.onnx", "wb") as f:
    f.write(onx.SerializeToString())

ValueError: ignored

In [58]:
x_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 225000 entries, 132065 to 423436
Data columns (total 12 columns):
 #   Column                           Non-Null Count   Dtype 
---  ------                           --------------   ----- 
 0   Year                             225000 non-null  object
 1   Quarter                          225000 non-null  object
 2   Month                            225000 non-null  object
 3   DayofMonth                       225000 non-null  object
 4   DayOfWeek                        225000 non-null  object
 5   DOT_ID_Reporting_Airline         225000 non-null  object
 6   Flight_Number_Reporting_Airline  225000 non-null  object
 7   OriginAirportID                  225000 non-null  object
 8   OriginState                      225000 non-null  object
 9   DestAirportID                    225000 non-null  object
 10  DestState                        225000 non-null  object
 11  DepTime                          225000 non-null  object
dtypes: object(1